<img src="img/dsci513_header2.png" width="600">

# Lab 4: MongoDB query language

## Instructions
---
rubric={mechanics:2}

- Follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/)
- Make sure to upload a PDF version of your lab notebook to Gradescope, in addition to the `.ipynb` file.
- Add a link to your GitHub repository here: https://github.ubc.ca/MDS-2022-23/DSCI_513_lab4_bibianna.git

## Getting set up
---

In [1]:
!pip install pymongo
from pymongo import MongoClient
import json

with open('data/credentials_mongodb.json') as f:
    login = json.load(f)

client = MongoClient(**login)

## Exercise 1: Getting to know your MongoDB databases
---

### 1.1

rubric={accuracy:1}

List the databases that exist on your MongoDB Atlas cluster after loading sample databases. You can do this either by checking out the databases using Compass or by using `pymongo`'s `.list_database_names()` method.

In [2]:
client.list_database_names()

['sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_guides',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'admin',
 'local']

### 1.2

rubric={accuracy:1}

List the collections stored in the `sample_mflix` and `sample_airbnb` databases. You can do this either by checking out the databases using Compass or by using `pymongo`'s `.list_collection_names()` method.

In [3]:
client['sample_airbnb'].list_collection_names()

['listingsAndReviews']

In [4]:
client['sample_mflix'].list_collection_names()

['theaters', 'movies', 'sessions', 'users', 'comments']

### 1.3

rubric={accuracy:2}

We would like to create a summary report to get the number of documents in each collection of each database on our cluster. Write a python loop that produces an output similar to the following:

```
Database: sample_airbnb
(collection, n_docs) =  ('listingsAndReviews', 5555)

Database: sample_analytics
(collection, n_docs) =  ('transactions', 1746)
(collection, n_docs) =  ('accounts', 1746)
(collection, n_docs) =  ('customers', 500)
.
.
.
```

In previous questions, you've been introduced to the two methods of `pymongo` which are used to return database and collection names. Moreover, you can use the `.count_documents(filter={}))` method to count all documents in each collection.

**Note:** Skip `admin` and `local` databases in your loop.

In [5]:
for database in client.list_database_names():
    if database not in ["admin", "local"]:
        print(f"Database: {database}")
        for collection in client[database].list_collection_names():
            print(f"(collection, n_docs) = {collection}, {client[database][collection].count_documents(filter={})}")


Database: sample_airbnb
(collection, n_docs) = listingsAndReviews, 5555
Database: sample_analytics
(collection, n_docs) = transactions, 1746
(collection, n_docs) = accounts, 1746
(collection, n_docs) = customers, 500
Database: sample_geospatial
(collection, n_docs) = shipwrecks, 11095
Database: sample_guides
(collection, n_docs) = planets, 8
Database: sample_mflix
(collection, n_docs) = theaters, 1564
(collection, n_docs) = movies, 23530
(collection, n_docs) = sessions, 1
(collection, n_docs) = users, 185
(collection, n_docs) = comments, 41079
Database: sample_restaurants
(collection, n_docs) = restaurants, 25359
(collection, n_docs) = neighborhoods, 195
Database: sample_supplies
(collection, n_docs) = sales, 5000
Database: sample_training
(collection, n_docs) = routes, 66985
(collection, n_docs) = zips, 29470
(collection, n_docs) = trips, 10000
(collection, n_docs) = inspections, 80047
(collection, n_docs) = posts, 500
(collection, n_docs) = companies, 9500
(collection, n_docs) = grad

### 1.4

rubric={accuracy:1}

The _Schema_ tab of the Compass application provides a nice schema analyzer tool that helps to quickly obtain summary information about collections in a database both visually and in text format.

Suppose that we would like to find out how high-rated movies/TV series on _metacritic_ are rated on IMDB. Use the schema analyzer tool to find the distribution of the `imdb.rating` field for movies/TV series in the `sample_mflix` database that have a `metacritic` rating of greater than 90. Paste a screenshot of the distribution that you find here.

**Hint:** There are 111 documents with a `metacritic` rating of greater than 90.

## Exercise 2: Basic MongoDB queries
---

### 2.1

rubric={accuracy:1}

Retrieve one (random) document associated with some movie/TV series produced in 2015.

You can use `.find_one()` method to do this, or use `.find()` but limit your results to 1 document.

In [6]:
list(
    client['sample_mflix']['movies'].find(filter={'year' : 2015}, limit=1)

)

[{'_id': ObjectId('573a13adf29313caabd2b765'),
  'plot': "A new theme park is built on the original site of Jurassic Park. Everything is going well until the park's newest attraction--a genetically modified giant stealth killing machine--escapes containment and goes on a killing spree.",
  'genres': ['Action', 'Adventure', 'Sci-Fi'],
  'runtime': 124,
  'metacritic': 59,
  'rated': 'PG-13',
  'cast': ['Chris Pratt',
   'Bryce Dallas Howard',
   'Irrfan Khan',
   "Vincent D'Onofrio"],
  'num_mflix_comments': 0,
  'poster': 'https://m.media-amazon.com/images/M/MV5BNzQ3OTY4NjAtNzM5OS00N2ZhLWJlOWUtYzYwZjNmOWRiMzcyXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_SY1000_SX677_AL_.jpg',
  'title': 'Jurassic World',
  'fullplot': '22 years after the original Jurassic Park failed, the new park (also known as Jurassic World) is open for business. After years of studying genetics the scientists on the park genetically engineer a new breed of dinosaur. When everything goes horribly wrong, will our heroes make it 

### 2.2

rubric={accuracy:1}

Retrieve all TV series produced in 1995.

> **Hint:** Inspect possible values for the `type` field.

In [7]:
list(
    client['sample_mflix']['movies'].find(filter={'type':'series', 'year':1995}, limit=1)
)

[{'_id': ObjectId('573a1399f29313caabceee5e'),
  'plot': 'Captain Woodrow Call, now retired from the Rangers, is a bounty hunter. He is hired by an eastern rail baron to track down Joey Garza, a new kind of killer, only a boy, who kills from a ...',
  'genres': ['Drama', 'Western'],
  'runtime': 300,
  'cast': ['James Garner', 'Sissy Spacek', 'Sam Shepard', 'Ned Beatty'],
  'poster': 'https://m.media-amazon.com/images/M/MV5BMjA5MTc2NjU5NF5BMl5BanBnXkFtZTcwODUzNjUyMQ@@._V1_SY1000_SX677_AL_.jpg',
  'title': 'Streets of Laredo',
  'fullplot': "Captain Woodrow Call, now retired from the Rangers, is a bounty hunter. He is hired by an eastern rail baron to track down Joey Garza, a new kind of killer, only a boy, who kills from a distance with a rifle. Joined by his old compadre Pea Eye, it is a long ride to south Texas and the Mexican side of the border, where the past, in the form of Maria Garza, Joey's mother, haunts Call.",
  'languages': ['English'],
  'released': datetime.datetime(1995,

### 2.3

rubric={accuracy:2}

Retrieve the title and cast of movies/TV series produced in 2010, but limit your results to 5 documents.

> **Note:** Don't forget to exclude the `_id` field from your returned documents.

> **Note:** The exact returned documents returned by `pymongo` might be different in different sessions and on different computers. This is perfectly fine.

In [8]:
list(
    client['sample_mflix']['movies'].find(filter={'year':2010}, 
                                         projection={'title': 1, 'cast':1, '_id':0},
                                        limit=5)
)

[{'cast': ['èva Gèbor', 'Istvèn Znamenèk', 'èkos Horvèth', 'Lia Pokorny'],
  'title': 'Pèl Adrienn'},
 {'title': 'In My Sleep',
  'cast': ['Philip Winchester',
   'Tim Draxl',
   'Lacey Chabert',
   'Abigail Spencer']},
 {'cast': ['James Badge Dale',
   'Joseph Mazzello',
   'Jon Seda',
   'Sebastian Bertoli'],
  'title': 'The Pacific'},
 {'cast': ['Mandy Moore', 'Zachary Levi', 'Donna Murphy', 'Ron Perlman'],
  'title': 'Tangled'},
 {'cast': ['Nikita Mikhalkov',
   'Oleg Menshikov',
   'Nadezhda Mikhalkova',
   'Sergey Makovetskiy'],
  'title': 'Utomlyonnye solntsem 2: Predstoyanie'}]

### 2.4

rubric={accuracy:2}

Retrieve the top 15 movies produced in 2010 that have the longest duration. Exclude TV series from your results. The returned documents should only contain the `title` and `runtime` fields (exclude the `_id` field).

> **Note:** It's ok if your results contain duplicate movies.

In [9]:

list(
    client['sample_mflix']['movies'].find(
        filter={'year': 2010, 'type':'movie'},
        projection={'_id': 0, 'title': 1, 'runtime': 1},
        limit=15,
        sort=[('runtime', -1)]
    )
)

[{'runtime': 272, 'title': 'Mysteries of Lisbon'},
 {'runtime': 240, 'title': 'Never Sleep Again: The Elm Street Legacy'},
 {'runtime': 181, 'title': 'Utomlyonnye solntsem 2: Predstoyanie'},
 {'runtime': 181, 'title': 'Aurora'},
 {'runtime': 180, 'title': 'Thorne: Sleepyhead'},
 {'runtime': 180, 'title': 'The Autobiography of Nicolae Ceausescu'},
 {'runtime': 178, 'title': 'Riverworld'},
 {'runtime': 174, 'title': 'Enthiran'},
 {'runtime': 170, 'title': 'Khaleja'},
 {'runtime': 170, 'title': 'We Believed'},
 {'title': 'My Name Is Khan', 'runtime': 165},
 {'runtime': 163, 'title': 'Raajneeti'},
 {'runtime': 163, 'title': 'Moss'},
 {'runtime': 160, 'title': 'Singam'},
 {'runtime': 159, 'title': 'Black Venus'}]

### 2.5

rubric={accuracy:2}

For each year between 2010 and 2015 (inclusive), return the number of movies/TV series with a metacritic rating of exactly 90. Your results should look like this:

```
2010: 2
2011: 2
2012: 1
2013: 2
2014: 1
2015: 3
```

In [10]:
for y in range(2010,2016):
    count = client['sample_mflix']['movies'].count_documents(filter={'year':y, 'metacritic' : 90})
    print(str(y) + ': ' + str(count))

2010: 2
2011: 2
2012: 1
2013: 2
2014: 1
2015: 3


In [11]:
list(
    client["sample_mflix"]["movies"].aggregate(
        [
            {'$group': {
                '_id': '$type',
                'count_each_type': {'$sum': 1}
            }}
        ]
    )
)

[{'_id': 'movie', 'count_each_type': 23276},
 {'_id': 'series', 'count_each_type': 254}]

### 2.6

rubric={accuracy:2}

Retrieve the title and runtime of the 10 shortest movies in the `movies` collection.

For this exercise, you need to make sure that the field `runtime` exists in the returned documents, otherwise by default those documents would appear first which don't have a `runtime` field at all!

> **Hint:** You need the `$exists` operator (see [here](https://docs.mongodb.com/manual/reference/operator/query/exists/) for help).

In [12]:

list(
    client['sample_mflix']['movies'].find(
        filter={'type':'movie',
               'runtime': {'$exists':True}},
        projection={'_id': 0, 'title': 1, 'runtime': 1},
        limit=10,
        sort=[('runtime', 1)]
    )
)

[{'runtime': 1, 'title': 'Neko no shukai'},
 {'runtime': 1, 'title': 'Newark Athlete'},
 {'runtime': 1, 'title': 'Blacksmith Scene'},
 {'runtime': 1, 'title': 'The Kiss'},
 {'runtime': 1, 'title': 'Dickson Experimental Sound Film'},
 {'runtime': 1, 'title': 'The Kiss'},
 {'runtime': 2, 'title': 'Andrè and Wally B.'},
 {'runtime': 2, 'title': 'Luxo Jr.'},
 {'runtime': 2, 'title': 'Game Over'},
 {'runtime': 2, 'title': 'Fresh Guacamole'}]

### 2.7

rubric={accuracy:2}

Retrieve documents for movies/TV series whose title contains two digits at the end, separated by a space (e.g. "Apollo 13").

- Your results should only contain the `title` and `year` fields
- Sort the documents first by production year in descending order, and then alphabetically by title in ascending order.
- Limit your results to 15 documents.

> **Note:** It's ok if your results contain duplicates.

In [13]:
list(
    client['sample_mflix']['movies'].find(
        filter={'title':{'$regex' : ' \d{2}$'}},
        projection={'_id': 0, 'title': 1, 'year': 1},
        limit=15,
        sort=[('title', -1), ('year', 1)]
    )
)

[{'title': 'Zone 39', 'year': 1996},
 {'title': 'Your Life in 65', 'year': 2006},
 {'title': 'Ward 13', 'year': 2003},
 {'title': 'United 93', 'year': 2006},
 {'title': 'Truth in 24', 'year': 2008},
 {'title': 'Track 29', 'year': 1988},
 {'title': 'Torremolinos 73', 'year': 2003},
 {'title': 'Tony 10', 'year': 2012},
 {'title': 'This Is 40', 'year': 2012},
 {'title': 'The Perfect 46', 'year': 2014},
 {'title': 'The Number 23', 'year': 2007},
 {'title': 'The 33', 'year': 2015},
 {'title': 'Symphony No. 42', 'year': 2014},
 {'year': 2012, 'title': 'Storage 24'},
 {'year': 2006, 'title': 'Starter for 10'}]

## Exercise 3: Conditionals, embedded documents & arrays
---

### 3.1

rubric={accuracy:2}

Retrieve the title, production year, and number of awards of all movies that

- have been produced between 1950 and 2000 (inclusive)
- have an IMDB rating of 8.5 or better
- won at least 30 awards.

Sort the results by production year in descending order.

In [14]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            '$and': [   
                {'year': {'$gte' : 1950, '$lte' : 2000}},
                {'imdb.rating' : {'$gte' : 8.5}},
                {'awards.wins': {'$gte' : 30}}
            ]
        },
        projection={"_id": 0, "title" : 1, "year": 1, "awards.wins":1},
        sort=[('year', -1)]
    )
)

[{'year': 2000, 'title': 'Gladiator', 'awards': {'wins': 63}},
 {'year': 2000, 'title': 'Memento', 'awards': {'wins': 54}},
 {'year': 1999, 'title': 'The Matrix', 'awards': {'wins': 37}},
 {'year': 1998, 'title': 'Saving Private Ryan', 'awards': {'wins': 83}},
 {'year': 1997, 'title': 'Life Is Beautiful', 'awards': {'wins': 66}},
 {'title': 'Life Is Beautiful', 'awards': {'wins': 66}, 'year': 1997},
 {'year': 1995, 'title': 'Se7en', 'awards': {'wins': 32}},
 {'year': 1995, 'title': 'The Usual Suspects', 'awards': {'wins': 36}},
 {'year': 1994, 'title': 'Forrest Gump', 'awards': {'wins': 46}},
 {'year': 1994, 'title': 'Pulp Fiction', 'awards': {'wins': 64}},
 {'title': "Schindler's List", 'awards': {'wins': 81}, 'year': 1993},
 {'year': 1991, 'title': 'The Silence of the Lambs', 'awards': {'wins': 56}},
 {'year': 1990, 'title': 'Goodfellas', 'awards': {'wins': 43}},
 {'year': 1981, 'title': 'Raiders of the Lost Ark', 'awards': {'wins': 32}},
 {'year': 1977,
  'title': 'Star Wars: Episod

### 3.2

rubric={accuracy:2}

Find the top 15 highest-rated movies according to IMDB for movies that have at least 100,000 votes. Your returned documents should only contain the `title`, `year`, and `imdb.rating` fields.

> **Hint:** Be careful about documents which have a blank space in their `imdb.rating` field!

> **Note:** It's ok if your results contain duplicates. Return 15 documents in any case.

In [15]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            '$and': [   
                {'imdb.votes': {'$gte' : 100000}},
                {'imdb.rating' : {'$exists' : True}},
            ]
        },
        projection={"_id": 0, "title" : 1, "year": 1, "imdb.rating":1},
        limit=15,
        sort=[('imdb.rating', -1)]
    )
)

[{'title': 'Band of Brothers', 'year': 2001, 'imdb': {'rating': 9.6}},
 {'imdb': {'rating': 9.3}, 'year': 1994, 'title': 'The Shawshank Redemption'},
 {'imdb': {'rating': 9.3}, 'year': 1994, 'title': 'The Shawshank Redemption'},
 {'imdb': {'rating': 9.2}, 'year': 1972, 'title': 'The Godfather'},
 {'imdb': {'rating': 9.1}, 'year': 1974, 'title': 'The Godfather: Part II'},
 {'imdb': {'rating': 9.0}, 'year': 2008, 'title': 'The Dark Knight'},
 {'title': "Schindler's List", 'year': 1993, 'imdb': {'rating': 8.9}},
 {'imdb': {'rating': 8.9},
  'year': 2003,
  'title': 'The Lord of the Rings: The Return of the King'},
 {'imdb': {'rating': 8.9}, 'year': 1994, 'title': 'Pulp Fiction'},
 {'title': '12 Angry Men', 'year': 1957, 'imdb': {'rating': 8.9}},
 {'imdb': {'rating': 8.9},
  'year': 1966,
  'title': 'The Good, the Bad and the Ugly'},
 {'imdb': {'rating': 8.9}, 'year': 1999, 'title': 'Fight Club'},
 {'imdb': {'rating': 8.8},
  'year': 2001,
  'title': 'The Lord of the Rings: The Fellowship 

### 3.3

rubric={accuracy:2}

There is one **thriller** movie in the `movies` collection which has a single digit within a word in its title (e.g. "one2one"). Return the document associated with that movie. Only include the `title`, `year`, `genres`, and `cast` fields.

In [16]:
list(
    client["sample_mflix"]["movies"].find(
        filter= {
            'genres': 'Thriller',
            'title' : {'$regex' : '[a-zA-Z]+\d{1}[a-zA-Z]+'}
        },
        projection={"_id": 0, "title" : 1, "year": 1, "genres":1}
    )
)

[{'year': 1995, 'genres': ['Drama', 'Mystery', 'Thriller'], 'title': 'Se7en'}]

### 3.4

rubric={accuracy:1}

Retrieve the title, production year, and IMDB rating of movies in which both **Morgan Freeman** and **Clint Eastwood** played a role (among other actors in those movies). Sort the returned documents by year in descending order.

In [17]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            'cast': {'$all': ['Morgan Freeman', 'Clint Eastwood']}
        },
        projection={'_id': 0, 'title': 1, 'imdb.rating': 1},
        sort=[('year', 1)]
    )
)

[{'imdb': {'rating': 8.3}, 'title': 'Unforgiven'},
 {'imdb': {'rating': 8.1}, 'title': 'Million Dollar Baby'}]

### 3.5

rubric={accuracy:3}

Retrieve documents associated with movies which:

- are available in both German and French (among other languages), but not in English,
- are either rated above 8 according to IMDB, or above 7.5 according to the critic ratings of [Rotten Tomatoes](https://www.rottentomatoes.com/) (inspect the `tomatoes` field),
- have at least 50 Rotten Tomatoes critic reviews.

The returned documents should include the title, year, IMDB rating, Rotten Tomatoes critic rating and country of production fields. Sort the results by IMDB rating in descending order.

**Note:** Duplicates in the results are ok.

In [20]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            '$and' :
            [{'languages' : { '$all' : ['German', 'French']}},
             {'languages' : {'$nin' : ['English']}}
            ],
            '$or' : 
            [{'imdb.rating' : {'$gt' : 8}},
             {'tomatoes.critic.rating' : { '$gt' : 7.5}}
            ],
            'tomatoes.critic.numReviews' : { '$gte' : 50}
        },
    )
)

[{'_id': ObjectId('573a1394f29313caabce095d'),
  'fullplot': 'Florence Carala and her lover Julien Tavernier, an ex - paratrooper want to murder her husband by faking a suicide. But after Julien has killed him and he puts his things in his car, he finds he has forgotten the rope outside the window and he returns to the building to remove it...',
  'imdb': {'rating': 8.0, 'votes': 12431, 'id': 51378},
  'year': 1958,
  'plot': 'A self-assured business man murders his employer, the husband of his mistress, which unintentionally provokes an ill-fated chain of events.',
  'genres': ['Crime', 'Drama', 'Thriller'],
  'rated': 'NOT RATED',
  'metacritic': 92,
  'title': 'Elevator to the Gallows',
  'lastupdated': '2015-08-30 00:23:59.180000000',
  'languages': ['French', 'German'],
  'writers': ['Roger Nimier (adaptation)',
   'Louis Malle (adaptation)',
   'Roger Nimier (dialogue)',
   'Noèl Calef (novel)',
   'Noèl Calef (pre-adaptation)'],
  'type': 'movie',
  'tomatoes': {'viewer': {'rati

### 3.6

rubric={accuracy:2}

Find the title and production year of the top 20 award-winning movies which have **not** been produced in USA, Canada, UK, or Australia.

**Note:** Duplicates are ok. Return 20 documents in any case.

## Exercise 4: Aggregation pipelines
---

For this exercise, my suggestion is for you to create your pipelines in Compass. Once you're happy with the results, you can use the "Export pipeline code to language" button (right next to the green SAVE button) to get the Python code for your pipeline, and use it here in the lab's Jupyter Notebook. One good thing about constructing pipelines in Compass is that you can also see intermediate results after each stage.

If your comfortable with `pymongo`, feel free to write your pipelines directly here.

### 4.1

rubric={accuracy:2}

Let's start by creating a simple pipeline that does a similar job as Exercise [3.6](#3.6):

> Find the title and production year of the top 20 award-winning movies which have **not** been produced in USA, Canada, UK, or Australia.

In addition, promote the `imdb.rating` subfield to show up as an independent field named `imdbRating` in the resulting documents.

Don't forget to check if you get the same movies as in Exercise [3.6](#3.6)!

In [ ]:
pipeline = [
    { "$match": {
        "countries": { "$nin": ["USA", "Canada", "UK", "Australia"] }
    }},
    { "$sort": { "numAwards": -1 }},
    # limit to the top 20 movies
    { "$limit": 20 },
    # project only the title and production year
    { "$project": {
        "title": 1,
        "year": 1
    }}
]

# run the aggregation pipeline and store the results in a cursor
cursor = db.movies.aggregate(pipeline)

# iterate over the cursor and print the results
for doc in cursor:
    print(doc)


### 4.2

rubric={reasoning:1}

Does the order of stages in the previous question matter? Provide an example and explain why.

_Your answer goes here_

### 4.3

rubric={accuracy:2}

Count the number of movies (not TV series) produced in the USA per year, in and after the year 2000. Sort your results by year in ascending order.

Your results should look like this:

```
[{'_id': 2000, 'num_of_movies': 341},
 {'_id': 2001, 'num_of_movies': 318},
 {'_id': 2002, 'num_of_movies': 337},
.
.
.
]
```

In [ ]:
...

### 4.4

rubric={accuracy:1}

Can you make the output of the previous exercise look like this?

```
[{'num_of_movies': 341, 'prod_year': 2000},
 {'num_of_movies': 318, 'prod_year': 2001},
 {'num_of_movies': 337, 'prod_year': 2002},
.
.
.
]
```

**Note:** The order in which different fields appear in the results does not matter.

In [ ]:
...

### 4.5

rubric={accuracy:2}

Create an aggregation pipeline that lists the top 10 most common movie languages, along with the count of movies for each language. Sort your results based on the count in descending order.

Your results should look like this:

```
[{'_id': 'English', 'num_of_movies': 16048},
 {'_id': 'French', 'num_of_movies': 2800},
 {'_id': 'Spanish', 'num_of_movies': 1787},
 .
 .
 .
 ]
 ```
 
> **Hint:** Languages are listed in an array, so they need special treatment before grouping.

In [ ]:
...

### 4.6

rubric={accuracy:3}

Create an aggregation pipeline that lists the top 15 busiest directors along with the number of movies they have directed. Your pipeline should also return the average IMDB rating of the movies directed by each director, **rounded to 1 decimal digit**.

Your results should look like this:

```
[{'_id': 'Woody Allen', 'num_of_movies': 40, 'avg_imdb_rating': 7.2},
 {'_id': 'John Ford', 'num_of_movies': 35, 'avg_imdb_rating': 7.2},
 {'_id': 'Takashi Miike', 'num_of_movies': 34, 'avg_imdb_rating': 6.9},
 .
 .
 .
 ]
```
 
> **Hint:** Directors are listed in an array, so they need special treatment before grouping.

> **Hint:** You can use the `$round` operator in a projection stage to round the average IMDB rating values (see [here](https://docs.mongodb.com/manual/reference/operator/aggregation/round/) for help).

In [ ]:
...

## Exercise 5: But I only speak SQL!
---

Recently, a software engineer has joined your team who is well-seasoned in SQL, but has never used MongoDB. Your supervisor knows that you've had some experience with MongoDB in the MDS program, so she asks you to help translate the SQL queries written by the new software engineer to MongoDB queries. Your MongoDB queries will be used in a NoSQL-based application developed by your company.

In the following questions, imagine that each document is a row in a table, and each field represents a column. Also, imagine that in our hypothetical table, documents with array fields appear as multiple rows, each row having one of the array elements in that particular array field.

Try to translate each SQL query to its equivalent MongoDB query as if your collection was a table (i.e. relation) in a relational database.

### 5.1

rubric={accuracy:3}

```sql
SELECT
    countries, COUNT(*) AS count_movies
FROM
    movies
WHERE
    imdb_rating > 8
GROUP BY
    countries
ORDER BY
    count_movies DESC
LIMIT
    20
```

In [ ]:
...

### 5.2

rubric={accuracy:2}

```sql
SELECT
    directors, MAX(awards_wins) AS max_awards_wins
FROM
    movies
GROUP BY
    directors
ORDER BY
    max_awards_wins DESC
LIMIT
    10
```

In [ ]:
...

### 5.3

Well, I don't feel good about making you think of NoSQL databases in terms of the relations, rows, and columns. But I really want to make it up to you, so I'm hoping you'll find this one funny:

<img src="img/nosql_cartoon.jpeg" width="600">

([image source](https://www.deviantart.com/harshadpd/art/Relational-Issues-719533744))